In [11]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import os, sys
# Add the parent directory (one level up) to the Python path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.insert(0, parent_dir)

import warnings
warnings.filterwarnings("ignore")

from modules.inference import preprocessing, object_detection, nms

In [21]:
#predict using model
def yolo_predict(model, image, score_threshold = 0.5, nms_iou_threshold = 0.5):

    #predict from model
    predicted_outputs = model.predict(image)
    bounding_boxes, class_categories, scores = model.post_processv2(predicted_outputs, score_threshold = score_threshold)

    max_scores = [x[np.argmax[x]] for x in scores]

    #nms
    indices = nms.filter(bounding_boxes, max_scores, score_threshold=0.0, nms_iou_threshold=nms_iou_threshold)

    filtered_boxes = [bounding_boxes[x] for x in indices]
    filtered_scores = [scores[x] for x in indices]
    filtered_categories = [class_categories[x] for x in indices]    


    return filtered_boxes, filtered_categories, filtered_scores

In [22]:
#load model 1 and model 2
yolo_model1 = object_detection.Model()
yolo_model2 = object_detection.Model('../storage/models/yolo_model_2/yolov4-tiny-logistics_size_416_2.weights', 
                               '../storage/models/yolo_model_2/yolov4-tiny-logistics_size_416_2.cfg',
                               '../storage/models/yolo_model_2/logistics.names')

In [23]:
destination_dir = '../storage/data/dataset_predictions'
data_dir = '../storage/data/dataset'
data_paths = os.listdir(data_dir)
image_paths = [os.path.join(data_dir, x) for x in data_paths if x.endswith('.jpg')]

In [24]:
#laod image
image = cv2.imread(image_paths[0])

#preprocess image using the same pipeline used before
image_processed = cv2.dnn.blobFromImage(image,
                    scalefactor = 1/255.,
                    size = (416, 416),
                    mean = (0, 0, 0),
                    swapRB = True,
                    crop = False
)

#get yolo 1 to predict
filtered_boxes2, filtered_categories2, filtered_scores2 = yolo_predict(yolo_model2, image_processed)


TypeError: 'function' object is not subscriptable

In [8]:
def save_predictions(filename, filtered_boxes, filtered_categories, filtered_scores):
    
    filename = filename
    with open(filename, 'w') as file:
        for i, id in enumerate(filtered_categories):
                file.write(f"{id}")

                for box in filtered_boxes[i]:
                     file.write(f" {box}")
                file.write(" " + " ".join(str(score) for score in filtered_scores[i]))
                file.write(f"\n")




In [9]:
save_predictions('test.txt', bounding_boxes, class_categories, scores)

In [10]:
#save scores for the first 1000 image sin the techtrack dataset
for image_path in image_paths[:1]:

    #laod image
    image = cv2.imread(image_path)

    #preprocess image using the same pipeline used before
    image_processed = cv2.dnn.blobFromImage(image,
                        scalefactor = 1/255.,
                        size = (416, 416),
                        mean = (0, 0, 0),
                        swapRB = True,
                        crop = False
    )

    predicted_outputs = yolo_model2.predict(image_processed)
    filtered_boxes2, filtered_categories2, filtered_scores2 = yolo_model2.post_processv2(predicted_outputs, score_threshold = 0.5)
           
    #nms
    indices = nms.filter(filtered_boxes2, filtered_scores2, score_threshold=0.0, nms_iou_threshold=0.25)

    filtered_boxes = [bounding_boxes[x] for x in indices]
    filtered_scores = [scores[x] for x in indices]
    filtered_categories = [class_categories[x] for x in indices]    


    #filenmae
    file_name = image_path.split('/')[-1].replace('.jpg', '_pred.txt')
    dest_path = os.path.join(destination_dir, file_name)
    dest_path = 'test.txt'
    save_predictions(dest_path, filtered_boxes2, filtered_categories2, filtered_scores2)

TypeError: Can't parse 'scores'. Sequence item with index 0 has a wrong type